In [1]:
import os
import pickle
import numpy as np
import datetime
import json
from multiprocessing import Process
from multiprocess import Process

import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import VGG16, MobileNetV2, MobileNetV3Small

from models import *
from utils import regression_stats
from img_utils import data_to_df, preprocess_images, set_gpu, set_cpu

import matplotlib.pyplot as plt

2024-04-24 16:24:34.503076: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
from img_utils import data_to_df
#try reloading the module
IMAGE_WIDTH: int = 448
IMAGE_HEIGHT: int = 448


# Load Data
train_1_path: str = "../nybolig-scrape/output/train/train_1"
train_2_path: str = "../nybolig-scrape/output/train/train_2"
valid_path: str = "../nybolig-scrape/output/valid"
test_path: str = "../nybolig-scrape/output/test"

train1_df, train2_df, valid_df, test_df = data_to_df(
    [train_1_path, train_2_path, valid_path, test_path], preprocess=True
)

# TODO: Would be better with this format
# train_images: np.array = preprocess_images(
#     train1_df["image_floorplan"], IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
# )


#### Train Set 1 ####
train1_features = train1_df.drop(columns=["image_floorplan", "price"])
train1_images: np.array = preprocess_images(
    train1_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
train1_prices: np.array = train1_df["price"].values


#### Train Set 2 ####
train2_features = train2_df.drop(columns=["image_floorplan", "price"])
train2_images: np.array = preprocess_images(
    train2_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
train2_prices: np.array = train2_df["price"].values


#### Validation Set ####
valid_features = valid_df.drop(columns=["image_floorplan", "price"])
valid_images: np.array = preprocess_images(
    valid_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
valid_prices: np.array = valid_df["price"].values


#### Test Set ####
test_features = test_df.drop(columns=["image_floorplan", "price"])
test_images: np.array = preprocess_images(
    test_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
test_prices: np.array = test_df["price"].values

Processing ../nybolig-scrape/output/train/train_1: 100%|██████████| 113/113 [00:00<00:00, 33081.34it/s]
Processing ../nybolig-scrape/output/train/train_2: 100%|██████████| 114/114 [00:00<00:00, 261427.37it/s]
Preprocessing: 100%|██████████| 4/4 [00:00<00:00, 47.97it/s]


In [6]:
def save_expected_predicted(test_prices, test_predictions, img_dir):
        #Set X and Y axis to [0, 9.000.000]
    #plt.xlim(0, 9999999)
    #plt.ylim(0, 9999999)
    plt.scatter(test_prices, test_predictions)
    plt.xlabel("Expected Price")
    plt.ylabel("Predicted Price")
    plt.title("Expected vs Predicted Price")
    try: 
        plt.plot([min(test_prices), max(test_prices)], [min(test_prices), max(test_prices)], color='red')
    except:
        pass
    plt.savefig(f"{img_dir}/expected_vs_predicted.png")
    plt.close()

def save_residuals(test_prices, test_predictions, img_dir):
    residuals = test_prices - test_predictions.reshape(-1)
    plt.scatter(test_predictions, residuals)
    try:
        plt.hlines(y=0, xmin=test_prices.min(), xmax=test_prices.max(), colors="r")
    except:
        pass
    plt.xlabel("Expected Price")
    plt.ylabel("Residuals")
    plt.title("Residuals")
    plt.savefig(f"{img_dir}/residuals.png")
    plt.close()

def get_saliency_map(model, image):
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    image = image.astype(np.float32)
    image = tf.convert_to_tensor(image)
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
    gradients = tape.gradient(prediction, image)
    gradients = tf.squeeze(gradients)
    gradients = tf.reduce_max(gradients, axis=-1)
    gradients = gradients.numpy()
    gradients = (gradients - np.min(gradients)) / (np.max(gradients) - np.min(gradients))
    return gradients

def save_worst_best_predictions(model, test_predictions, test_prices, test_images, img_dir):
    residuals = test_prices - test_predictions.reshape(-1)
    distances = np.abs(test_prices - test_predictions.reshape(-1))
    worst_predictions = np.argsort(distances)[-8:]
    best_predictions = np.argsort(distances)[:8]
    test_images = np.array(test_images)
    for i, idx in enumerate(worst_predictions):
        image = test_images[idx]
        price = test_prices[idx]
        prediction = test_predictions[idx]
        residual = residuals[idx]
        plt.imshow(image)
        textstr = '\n'.join((
            f"Price: {price}",
            f"Predicted Price: {prediction}",
            f"Residual: {residual}"
        ))
        plt.text(0.01, 0.99, textstr, fontsize=10, transform=plt.gcf().transFigure, verticalalignment='top')
        plt.axis("off")
        plt.savefig(f"{img_dir}/worst_{i}.png")
        plt.close()
        
        saliency_map = get_saliency_map(model, image)
        plt.imshow(saliency_map, cmap="hot")
        plt.axis("off")
        plt.savefig(f"{img_dir}/worst_saliency_map_{i}.png")
        plt.close()
        
    for i, idx in enumerate(best_predictions):
        image = test_images[idx]
        price = test_prices[idx]
        prediction = test_predictions[idx]
        residual = residuals[idx]
        plt.imshow(image)
        textstr = '\n'.join((
            f"Price: {price}",
            f"Predicted Price: {prediction}",
            f"Residual: {residual}"
        ))
        plt.text(0.01, 0.99, textstr, fontsize=10, transform=plt.gcf().transFigure, verticalalignment='top')
        plt.axis("off")
        plt.savefig(f"{img_dir}/best_{i}.png")
        plt.close()
        saliency_map = get_saliency_map(model, image)
        plt.imshow(saliency_map, cmap="hot")
        plt.axis("off")
        plt.savefig(f"{img_dir}/best_saliency_map_{i}.png")
        plt.close()

def save_features_importance(feature_importance, img_dir):
    #sort the feature_importance dict by value
    feature_importance = {k: v for k, v in sorted(feature_importance.items(), key=lambda item: item[1], reverse=True)}
    #add percentages to the bars
    plt.bar(feature_importance.keys(), feature_importance.values())
    #plt.bar_label = feature_importance.values()
    plt.title('Feature Importance')
    #Remove y-labels
    plt.ylabel('')
    plt.xticks(rotation=90)
    #Zoom out so that text is visible 
    plt.subplots_adjust(bottom=0.4)
    plt.savefig(f"{img_dir}/feature_importance.png")
    plt.close()

def save_worst_best(test_predictions, test_prices, test_features, model_dir):
    #Find the best predictions, and worst predictions. 
    #Save them in two dataframes. Save a latex of the dataframe in a txt-file 
    residuals = test_prices - test_predictions.reshape(-1)
    distances = np.abs(test_prices - test_predictions.reshape(-1))
    worst_predictions = np.argsort(distances)[-8:]
    best_predictions = np.argsort(distances)[:8]
    
    test_features_ = pd.DataFrame(test_features).copy()
    test_features_["Price"] = test_prices
    test_features_["Predicted Price"] = test_predictions
    test_features_["Residual"] = residuals
    test_features_ = test_features_.sort_values(by="Residual", ascending=False)
    worst_df = test_features_.iloc[worst_predictions]
    best_df = test_features_.iloc[best_predictions]
    #save worst and best as latex in txt-file 
    worst_df.to_latex(f"{model_dir}/worst_predictions.txt")
    best_df.to_latex(f"{model_dir}/best_predictions.txt")
    


def save_model_and_evaluate(
    model: object,
    fit_history: object,
    test_images: np.array,
    test_features: np.array,
    test_prices: np.array,
    model_dir: str,
    model_type:str
):
    if model_type == 'RF':
        print("Saving Model...")
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        with open(f"{model_dir}/model", "wb") as file_pi:
            pickle.dump(model, file_pi)
        test_predictions = model.predict(test_features)
    
    if model_type == "CNN":
        # Save Model
        print("Saving Model...")
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        model.save(f"{model_dir}/model")
        # Save Training History
        with open(f"{model_dir}/history", "wb") as file_pi:
            pickle.dump(fit_history.history, file_pi)
        test_predictions = model.predict(test_images)
        #Save Model Architecture
        #plot_model(model, to_file=f"{model_dir}/model_architecture.png", show_shapes=True, show_layer_names=True, show_dtype=True, rankdir="TB", expand_nested=False, dpi=96)
        img = plot_model(model, to_file=f"{model_dir}/architecture.png", show_shapes=True, show_layer_names=True, show_dtype=True, rankdir="TB", expand_nested=False, dpi=96)


    if model_type == 'CNN_RF':
        print("Saving Model...")
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        with open(f"{model_dir}/model", "wb") as file_pi:
            pickle.dump(model, file_pi)
        test_predictions = model.predict(test_images, test_features)
        

    # Evaluate Model
    print("Evaluating Model...")
    r2, mae, percentage_error, mse = regression_stats(test_prices, test_predictions)

    try:
        feature_importance = model.feature_importances_
        if model_type == "RF":
            feature_importance = dict(zip(test_features.columns, feature_importance))
    except AttributeError:
        print("Cant find feature_importance")
        feature_importance = None

    # Load existing evaluation data
    evaluation_file_path = f"{model_dir}/evaluation.json"
    evaluation_data = {}
    if os.path.exists(evaluation_file_path):
        with open(evaluation_file_path, "r") as json_file:
            evaluation_data = json.load(json_file)

    # Add new evaluation data
    new_evaluation = {
        "Timestamp": str(datetime.datetime.now()),
        "R2": r2,
        "MAE": mae,
        "Percentage Error": percentage_error,
        "MSE": mse,
        "Feature Importances": (feature_importance),
    }
    evaluation_data[len(evaluation_data)] = new_evaluation

    # Save updated evaluation data
    with open(evaluation_file_path, "w") as json_file:
        json.dump(evaluation_data, json_file, indent=4)

    # Compute median evaluation values from all instances
    r2_values = [evaluation_data[key]["R2"] for key in evaluation_data]
    mae_values = [evaluation_data[key]["MAE"] for key in evaluation_data]
    percentage_error_values = [
        evaluation_data[key]["Percentage Error"] for key in evaluation_data
    ]
    mse_values = [evaluation_data[key]["MSE"] for key in evaluation_data]

    median_evaluation_data = {
        "R2": np.median(r2_values),
        "MAE": np.median(mae_values),
        "Percentage Error": np.median(percentage_error_values),
        "MSE": np.median(mse_values),
    }

    with open(f"{model_dir}/median_evaluation.json", "w") as json_file:
        json.dump(median_evaluation_data, json_file, indent=4)

    print("\nModel Evaluation:")
    print(new_evaluation)
    print("\nMedian Evaluation:")
    print(median_evaluation_data)
    print("Feauter Importance...")
    print(feature_importance)

    # Images (Create or open existing folder)
    if not os.path.exists(f"{model_dir}/images"):
        os.makedirs(f"{model_dir}/images")
    img_dir = f"{model_dir}/images"
    
    save_expected_predicted(test_prices, test_predictions, img_dir)
    save_residuals(test_prices, test_predictions, img_dir)
    
    if model_type == 'CNN':
        print("\nSaving Best and Worst Image Predictions")
        save_worst_best_predictions(model, test_predictions, test_prices, test_images, img_dir)
    
    if model_type != 'CNN': 
        print("\nSaving Feature Importance")
        save_features_importance(feature_importance, img_dir)

    save_worst_best(test_predictions, test_prices, test_features, model_dir)
    print("\nDone!")




def train_save_model(
    model_func: object,
    args: tuple,
    test_images: np.array,
    test_features: np.array,
    test_prices: np.array,
    model_dir: str,
    use_gpu: bool,
    model_type:str
):
    if use_gpu:
        set_gpu()
    else:
        set_cpu()

    if model_type == "CNN":
        model, fit_history = model_func(*args)
    if model_type == 'RF':
        model = model_func(*args)
        fit_history = None
    if model_type == 'CNN_RF':
        model = model_func(*args)
        fit_history = None
    save_model_and_evaluate(model, fit_history, test_images, test_features, test_prices, model_dir, model_type)


def train_save_models(
    model_func: object,
    args: tuple,
    test_images: np.array,
    test_prices: np.array,
    model_dir: str,
    use_gpu: bool,
):
    if use_gpu:
        set_gpu()
    else:
        set_cpu()

    models, fit_histories = model_func(*args)
    for model_idx, (model, fit_history) in enumerate(zip(models, fit_histories)):
        save_model_and_evaluate(
            model, fit_history, test_images, test_prices, f"{model_dir}_{model_idx}"
        )

# Running on GPU

In [7]:
MODELS_PATH: str = "./models"
USE_GPU: bool = True

#TYPE = "RF"
# MODEL_NAME: str = "RF"
# FUNCTION: object = RF
# ARGS: tuple = (
#     train_images,
#     train_prices,
#     valid_images,
#     valid_prices,
# )
# p = Process(
#     target=train_save_model,
#     args=(FUNCTION, ARGS, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU),
# )



TYPE = "CNN"
MODEL_NAME: str = "MobileNetV2"
FUNCTION: object = CNN_model
ARGS: tuple = (
    MobileNetV2,
    True,
    train1_images,
    train1_prices,
    valid_images,
    valid_prices,
)
p = Process(
    target=train_save_model,
    args=(FUNCTION, ARGS, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU, TYPE),
)


# from models import CNN_RF_model
# TYPE = 'CNN_RF'
# MODEL_NAME: str = "MobileNetV2_RF"
# FUNCTION: object = CNN_RF_model
# ARGS: tuple = (
#     # keras.models.load_model(f"{MODELS_PATH}/MobileNetV2/model") #Load the model that is trained above
#     train2_images,
#     train2_features,
#     train2_prices,
# )
# p = Process(
#     target=train_save_model,
#     args=(FUNCTION, ARGS, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU, TYPE),
# )



# from models import CNN_AE_RF_model
#img_model =  keras.models.load_model(f"{MODELS_PATH}/MobileNetV2/model")
# TYPE = 'CNN_RF'
# MODEL_NAME: str = "MobileNetV2_AE_RF"
# FUNCTION: object = CNN_AE_RF_model
# ARGS: tuple = (
#      # keras.models.load_model(f"{MODELS_PATH}/MobileNetV2/model") #Load the model that is trained above (e.g. vgg16)
#     train2_images,
#     train2_features,
#     train2_prices,
# )
# p = Process(
#     target=train_save_models,
#     args=(FUNCTION, ARGS, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}"),
# )



# TYPE = 'CNN_RF'
# MODEL_NAME: str = "N_CNN_MobileNetV2_RF"
# FUNCTION: object = N_CNN_RF_model
# ARGS: tuple = (
#     2,
#     MobileNetV3Small,
#     np.concatenate((train1_images, train2_images), axis=0),
#     pd.concat((train1_features, train2_features), axis=0),

#     np.concatenate((train1_prices, train2_prices), axis=0),
# )
# p = Process(
#     target=train_save_models,
#     args=(FUNCTION, ARGS, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}"),
# )



#TYPE = "N_CNN"
# MODEL_NAME: str = "N_CNN_MobileNetV2"
# FUNCTION: object = N_CNN_model
# ARGS: tuple = (
#     MobileNetV2,
#     train_images,
#     train_prices,
#     valid_images,
#     valid_prices,
#     3,
# )
# p = Process(
#     target=train_save_models,
#     args=(FUNCTION, ARGS, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}"),
# )




p.start()
p.join()

Process Process-1:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.10/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
TypeError: train_save_model() missing 1 required positional argument: 'model_type'


# Running on CPU

In [8]:
%reload_ext autoreload
%autoreload 2
MODELS_PATH: str = "./models"
USE_GPU: bool = False

### RF

In [9]:
from models import RF
TYPE = "RF"
MODEL_NAME: str = "RF"
FUNCTION: object = RF
ARGS: tuple = (
    train2_features,
    train2_prices,
)
train_save_model(FUNCTION, ARGS, test_images, test_features, test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU, TYPE)

Setting CPU
Saving Model...
Evaluating Model...

Model Evaluation:
{'Timestamp': '2024-04-24 16:27:56.607591', 'R2': 0.8285589473024165, 'MAE': 966431.8352315673, 'Percentage Error': 26.22719955981217, 'MSE': 1988797196779.7188, 'Feature Importances': {'lattitude': 0.01705717524669862, 'longitude': 0.03229756548846307, 'postal_code': 0.10615194572626248, 'type': 0.0, 'postal_avg_sqm_price': 0.0, 'size': 0.6189949697329585, 'basement_size': 0.0, 'rooms': 0.18865932995915347, 'year_built': 0.013948218808430241, 'year_rebuilt': 0.0181661533729196, 'energy_label': 0.004724641665114032}}

Median Evaluation:
{'R2': 0.8241391743510718, 'MAE': 1004422.6693959701, 'Percentage Error': 27.814622804485744, 'MSE': 2040068650831.8735}
Feauter Importance...
{'lattitude': 0.01705717524669862, 'longitude': 0.03229756548846307, 'postal_code': 0.10615194572626248, 'type': 0.0, 'postal_avg_sqm_price': 0.0, 'size': 0.6189949697329585, 'basement_size': 0.0, 'rooms': 0.18865932995915347, 'year_built': 0.0139

### CNN

In [12]:
from models import CNN_model
TYPE = 'CNN'
MODEL_NAME: str = "MobileNetV2"
FUNCTION: object = CNN_model
ARGS: tuple = (
    MobileNetV3Small,
    True,
    train1_images,
    train1_prices,
    valid_images,
    valid_prices,
)
train_save_model(FUNCTION, ARGS, test_images, test_features, test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU, TYPE)

Setting CPU
Compiling Model
Fitting Model
Epoch 1/100
4/4 [==============================] - 34s 5s/step - loss: 3978696.5000 - mean_absolute_error: 3977040.7500 - val_loss: 3793892.7500 - val_mean_absolute_error: 3793022.5000
Epoch 2/100
4/4 [==============================] - 11s 3s/step - loss: 3975840.2500 - mean_absolute_error: 3975051.7500 - val_loss: 3789566.7500 - val_mean_absolute_error: 3788664.5000
Epoch 3/100
4/4 [==============================] - 12s 3s/step - loss: 3969618.2500 - mean_absolute_error: 3968606.2500 - val_loss: 3778251.0000 - val_mean_absolute_error: 3777033.7500
Epoch 4/100
4/4 [==============================] - 12s 3s/step - loss: 3954062.2500 - mean_absolute_error: 3952735.2500 - val_loss: 3751231.5000 - val_mean_absolute_error: 3749606.7500
Epoch 5/100
4/4 [==============================] - 11s 3s/step - loss: 3917852.5000 - mean_absolute_error: 3916042.5000 - val_loss: 3692288.7500 - val_mean_absolute_error: 3690033.2500
Epoch 6/100
4/4 [================

INFO:tensorflow:Assets written to: ./models/MobileNetV2/model/assets


2/2 [==============================] - 7s 1s/step
Evaluating Model...
Cant find feature_importance

Model Evaluation:
{'Timestamp': '2024-04-24 16:39:29.432211', 'R2': 0.19912610641197925, 'MAE': 1839526.6587301588, 'Percentage Error': 58.97405147893265, 'MSE': 9290515483193.63, 'Feature Importances': None}

Median Evaluation:
{'R2': -1.6735583622516519, 'MAE': 4406159.544906374, 'Percentage Error': 99.9500431825222, 'MSE': 31014539940164.57}
Feauter Importance...
None

Saving Best and Worst Image Predictions

Done!


### CNN RF

In [13]:
img_model =  keras.models.load_model(f"{MODELS_PATH}/MobileNetV2/model")

In [14]:
from models import CNN_RF_model
TYPE = 'CNN_RF'
MODEL_NAME: str = "MobileNetV2_RF"
FUNCTION: object = CNN_RF_model
ARGS: tuple = (
    img_model, # keras.models.load_model(f"{MODELS_PATH}/MobileNetV2/model")
    train2_images,
    train2_features,
    train2_prices,
)
train_save_model(FUNCTION, ARGS, test_images, test_features,  test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU, TYPE)

Setting CPU
4/4 [==============================] - 7s 1s/step
Saving Model...
2/2 [==============================] - 3s 1s/step
Evaluating Model...

Model Evaluation:
{'Timestamp': '2024-04-24 16:40:40.303149', 'R2': 0.82973645810032, 'MAE': 957930.6136137181, 'Percentage Error': 27.53683621360667, 'MSE': 1975137515290.366, 'Feature Importances': {'image_predictions': 0.05978953509743043, 'lattitude': 0.015862818708810372, 'longitude': 0.04504545153953138, 'postal_code': 0.10191371325505799, 'type': 0.0, 'postal_avg_sqm_price': 0.0, 'size': 0.5684099203648966, 'basement_size': 0.0, 'rooms': 0.1807754535835351, 'year_built': 0.01497920602653795, 'year_rebuilt': 0.00884813568336506, 'energy_label': 0.004375765740835261}}

Median Evaluation:
{'R2': 0.8223735192612573, 'MAE': 1007864.3283610474, 'Percentage Error': 28.00639476190426, 'MSE': 2060551083935.5537}
Feauter Importance...
{'image_predictions': 0.05978953509743043, 'lattitude': 0.015862818708810372, 'longitude': 0.0450454515395313

/usr/local/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(



Saving Feature Importance

Done!


### CN_AE_RF

In [15]:
from models import CNN_AE_RF_model
#img_model =  keras.models.load_model(f"{MODELS_PATH}/MobileNetV2/model")
TYPE = 'CNN_RF'
MODEL_NAME: str = "MobileNetV2_AE_RF"
FUNCTION: object = CNN_AE_RF_model
ARGS: tuple = (
    img_model, #keras.models.load_model(f"{MODELS_PATH}/MobileNetV2/model")
    train2_images,
    train2_features,
    train2_prices,
)
train_save_model(FUNCTION, ARGS, test_images, test_features,  test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU, TYPE)

Setting CPU
Epoch 1/10
4/4 [==============================] - 19s 3s/step - loss: 0.2018 - val_loss: 0.2048
Epoch 2/10
4/4 [==============================] - 12s 3s/step - loss: 0.1911 - val_loss: 0.1880
Epoch 3/10
4/4 [==============================] - 13s 3s/step - loss: 0.1730 - val_loss: 0.1618
Epoch 4/10
4/4 [==============================] - 12s 3s/step - loss: 0.1460 - val_loss: 0.1263
Epoch 5/10
4/4 [==============================] - 11s 3s/step - loss: 0.1113 - val_loss: 0.0868
Epoch 6/10
4/4 [==============================] - 11s 2s/step - loss: 0.0761 - val_loss: 0.0565
Epoch 7/10
4/4 [==============================] - 11s 3s/step - loss: 0.0538 - val_loss: 0.0446
Epoch 8/10
4/4 [==============================] - 11s 3s/step - loss: 0.0478 - val_loss: 0.0434
Epoch 9/10
4/4 [==============================] - 11s 3s/step - loss: 0.0488 - val_loss: 0.0440
Epoch 10/10
4/4 [==============================] - 5s 1s/step
Saving Model...
2/2 [==============================] - 3s 1s/s

/usr/local/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Evaluating Model...

Model Evaluation:
{'Timestamp': '2024-04-24 16:43:15.093784', 'R2': 0.8966876159070315, 'MAE': 833409.4302232856, 'Percentage Error': 26.599829111681313, 'MSE': 1198472458280.8247, 'Feature Importances': {'image_predictions': 0.0547240342187647, 'reconstruction_error': 0.04026456044299854, 'lattitude': 0.017320575169738495, 'longitude': 0.04238863562559668, 'postal_code': 0.09273245711900918, 'type': 0.0, 'postal_avg_sqm_price': 0.0, 'size': 0.6071024800545901, 'basement_size': 0.0, 'rooms': 0.12106811127167834, 'year_built': 0.011558257289309518, 'year_rebuilt': 0.010571116912140603, 'energy_label': 0.0022697718961739743}}

Median Evaluation:
{'R2': 0.8697556152145957, 'MAE': 934134.0543512926, 'Percentage Error': 26.92961149782996, 'MSE': 1510896388477.218}
Feauter Importance...
{'image_predictions': 0.0547240342187647, 'reconstruction_error': 0.04026456044299854, 'lattitude': 0.017320575169738495, 'longitude': 0.04238863562559668, 'postal_code': 0.09273245711900

### N-CNN

In [16]:
from models import N_CNN_RF_model
from keras.applications import MobileNetV3Small
TYPE = 'CNN_RF'
MODEL_NAME: str = "N_CNN_MobileNetV2_RF"
FUNCTION: object = N_CNN_RF_model
ARGS: tuple = (
    2,
    MobileNetV3Small,
    np.concatenate((train1_images, train2_images), axis=0),
    pd.concat((train1_features, train2_features), axis=0),

    np.concatenate((train1_prices, train2_prices), axis=0),
)
train_save_model(FUNCTION, ARGS, test_images, test_features,  test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU, TYPE)

Setting CPU


Compiling Model
Fitting Model
Epoch 1/100
4/4 [==============================] - 26s 4s/step - loss: 4100177.0000 - mean_absolute_error: 4098541.7500 - val_loss: 3974675.0000 - val_mean_absolute_error: 3973844.5000
Epoch 2/100
4/4 [==============================] - 13s 4s/step - loss: 4097649.2500 - mean_absolute_error: 4096912.5000 - val_loss: 3970960.7500 - val_mean_absolute_error: 3970133.5000
Epoch 3/100
4/4 [==============================] - 13s 3s/step - loss: 4092029.0000 - mean_absolute_error: 4091102.2500 - val_loss: 3960168.2500 - val_mean_absolute_error: 3959062.2500
Epoch 4/100
4/4 [==============================] - 13s 4s/step - loss: 4076274.5000 - mean_absolute_error: 4075067.5000 - val_loss: 3932723.0000 - val_mean_absolute_error: 3931239.0000
Epoch 5/100
4/4 [==============================] - 15s 4s/step - loss: 4038510.5000 - mean_absolute_error: 4036842.0000 - val_loss: 3870072.7500 - val_mean_absolute_error: 3867960.5000
Epoch 6/100
4/4 [============================

Compiling Model
Fitting Model
Epoch 1/100
4/4 [==============================] - 34s 5s/step - loss: 3976304.5000 - mean_absolute_error: 3974670.7500 - val_loss: 4099196.2500 - val_mean_absolute_error: 4098403.2500
Epoch 2/100
4/4 [==============================] - 15s 4s/step - loss: 3974827.0000 - mean_absolute_error: 3974155.0000 - val_loss: 4097955.7500 - val_mean_absolute_error: 4097257.0000
Epoch 3/100
4/4 [==============================] - 16s 5s/step - loss: 3973210.0000 - mean_absolute_error: 3972459.7500 - val_loss: 4094795.2500 - val_mean_absolute_error: 4093989.0000
Epoch 4/100
4/4 [==============================] - 15s 4s/step - loss: 3968725.2500 - mean_absolute_error: 3967885.2500 - val_loss: 4087048.0000 - val_mean_absolute_error: 4086078.0000
Epoch 5/100
4/4 [==============================] - 15s 4s/step - loss: 3958400.2500 - mean_absolute_error: 3957320.0000 - val_loss: 4069958.7500 - val_mean_absolute_error: 4068606.2500
Epoch 6/100
4/4 [============================

/usr/local/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(



Saving Feature Importance

Done!
